In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import Ollama
from langchain_ollama import OllamaLLM
from langchain.schema import Document
import numpy as np
from langchain_core.output_parsers import StrOutputParser

# === Datos de ejemplo ===
libros = [
    {"titulo": "1984", "descripcion": "Una novela distópica sobre un régimen totalitario que lo vigila todo."},
    {"titulo": "El nombre del viento", "descripcion": "Historia de Kvothe, un joven con un talento excepcional para la magia y la música."},
    {"titulo": "Dune", "descripcion": "Una epopeya de ciencia ficción sobre poder, religión y ecología en un planeta desértico."},
    {"titulo": "Un mundo feliz", "descripcion": "Sociedad futurista donde la estabilidad se consigue a costa de la libertad individual."},
]

# Crear documentos
documentos = [
    Document(page_content=libro["descripcion"], metadata={"titulo": libro["titulo"]})
    for libro in libros
]

# Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Vectorstore FAISS
vectorstore = FAISS.from_documents(documentos, embedding=embeddings)

llm = OllamaLLM(
    model="qwen2.5",
    base_url="http://localhost:11434",
    num_gpu_layers=90,       # Aceleración Metal máxima
    num_ctx=1536,            # Balance memoria/contexto
    temperature=0.5,
    )



In [20]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

retrieved_docs = retriever.invoke("")

for retrieved_doc in retrieved_docs:
    print()
    print(retrieved_doc)



page_content='Sociedad futurista donde la estabilidad se consigue a costa de la libertad individual.' metadata={'titulo': 'Un mundo feliz'}

page_content='Una novela distópica sobre un régimen totalitario que lo vigila todo.' metadata={'titulo': '1984'}

page_content='Historia de Kvothe, un joven con un talento excepcional para la magia y la música.' metadata={'titulo': 'El nombre del viento'}

page_content='Una epopeya de ciencia ficción sobre poder, religión y ecología en un planeta desértico.' metadata={'titulo': 'Dune'}


In [21]:
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)

template = """Eres un recomendador de libros
Tienes el context para responder a la Question 
Context: {context}

Question: {question}


Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

In [24]:
from operator import itemgetter

rag_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [28]:
rag_chain.invoke({"question": "Quiero un libro del que pueda llegar a ser similar a la cancion Wondewall de Oasis"})

'La canción "Wonderwall" de Oasis es una pieza emotiva y poética que habla sobre amor, esperanza y conexión. Basándome en tu solicitud, te recomendaría el libro **"El nombre del viento"** (título original en inglés: *The Name of the Wind*) de Patrick Rothfuss.\n\nEste libro combina elementos de narración épica con un tono poético que puede recordar a la profundidad emocional y la riqueza lingüística de "Wonderwall". La historia se centra en Kvothe, un personaje complejo y fascinante, similar al protagonista de la canción, cuya vida está llena de aventuras, magia y música.\n\nAunque no es una adaptación directa a la canción, el ambiente poético y la introspección emocional del libro podrían resonar contigo en un nivel similar.'